In [1]:
from os import environ
from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import seaborn as sns
import time
from riotwatcher import LolWatcher, ApiError
from sklearn.datasets import load_iris
import plotly.express as px

load_dotenv("../")

RIOT_API = environ["RIOT_API"]
lol_watcher = LolWatcher(RIOT_API)
my_region = "na1"
NA = "na1"
yogurtsauce = pd.DataFrame.from_dict(lol_watcher.summoner.by_name(NA, "yogurtsauce"), orient="index").T


In [2]:
ranked = lol_watcher.league.by_summoner(NA, yogurtsauce.id[0])
versions = lol_watcher.data_dragon.versions_for_region(NA)
champions_version = versions["n"]["champion"]
summoner_spells_version = versions["n"]["summoner"]
items_version = versions["n"]["item"]
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
my_matches = lol_watcher.match.matchlist_by_puuid(NA, yogurtsauce.puuid[0])

In [3]:
yogurtsauce

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q,jwWp9hL3BhaB8yVFHiTiKpYprW-dte-xemlv_M8cMXFD3Q,0_Z8M5TjaB96qj5ncZ3ewtq3H6CrLW6znsMYjxQ0Cgcrjr...,yogurtsauce,5028,1683450617805,511


In [4]:
class Summoner():
    def __init__(self, region, summoner_name):
        self.region = region
        self.name = summoner_name
        self.data = pd.DataFrame.from_dict(lol_watcher.summoner.by_name(self.region, self.name), orient="index").T
        self.id = self.data.id[0]
        self.account_id = self.data.accountId[0]
        self.puuid = self.data.puuid[0]
        self.profile_icon_id = self.data.profileIconId[0]
        self.revision_date = self.data.revisionDate[0]
        self.level = self.data.summonerLevel[0]
        self.matches = lol_watcher.match.matchlist_by_puuid(self.region, self.puuid)
        
yogurtsauce = Summoner("NA1", "yogurtsauce")

In [5]:
j = 0
cont = 0
last_match = yogurtsauce.matches[0]
match_detail = lol_watcher.match.by_id(NA, last_match)


In [23]:
match_detail["info"]["participants"][0]

{'allInPings': 0,
 'assistMePings': 0,
 'assists': 43,
 'baitPings': 0,
 'baronKills': 0,
 'basicPings': 0,
 'bountyLevel': 2,
 'challenges': {'12AssistStreakCount': 0,
  'abilityUses': 182,
  'acesBefore15Minutes': 5,
  'alliedJungleMonsterKills': 0,
  'baronTakedowns': 0,
  'blastConeOppositeOpponentCount': 0,
  'bountyGold': 300,
  'buffsStolen': 0,
  'completeSupportQuestInTime': 0,
  'controlWardsPlaced': 0,
  'damagePerMinute': 1965.308973060024,
  'damageTakenOnTeamPercentage': 0.18266474054558463,
  'dancedWithRiftHerald': 0,
  'deathsByEnemyChamps': 11,
  'dodgeSkillShotsSmallWindow': 2,
  'doubleAces': 0,
  'dragonTakedowns': 0,
  'earlyLaningPhaseGoldExpAdvantage': 0,
  'effectiveHealAndShielding': 839.800048828125,
  'elderDragonKillsWithOpposingSoul': 0,
  'elderDragonMultikills': 0,
  'enemyChampionImmobilizations': 0,
  'enemyJungleMonsterKills': 0,
  'epicMonsterKillsNearEnemyJungler': 0,
  'epicMonsterKillsWithin30SecondsOfSpawn': 0,
  'epicMonsterSteals': 0,
  'epicMo

In [29]:
N_GAMES = 5

games = {}
game_duration = np.zeros(N_GAMES)
damage = np.zeros(N_GAMES)

j = 0
cont = 0

while cont < N_GAMES:
    try:
        last_match = yogurtsauce.matches[cont]
        match_detail = lol_watcher.match.by_id(NA, last_match)
        participants = []
        
        for row in match_detail["info"]["participants"]:
            participants_row = {}
            participants_row["champion"] = row["championId"]
            participants_row["win"] = row["win"]
            participants_row["kills"] = row["kills"]
            participants_row["deaths"] = row["deaths"]
            participants_row["assists"] = row["assists"]
            participants_row["name"] = row["summonerName"]
            participants_row["level"] = row["summonerLevel"]
            
            participants_row["physical_dmg_taken"] = row["physicalDamageTaken"]
            participants_row["physical_dmg_dealt"] = row["physicalDamageDealt"]
            participants_row["physical_dmg_dealt_champ"] = row["physicalDamageDealtToChampions"]
            
            participants_row["magic_dmg_taken"] = row["magicDamageTaken"]
            participants_row["magic_dmg_dealt"] = row["magicDamageDealt"]
            participants_row["magic_dmg_dealt_champ"] = row["magicDamageDealtToChampions"]
            
            participants_row["true_dmg_taken"] = row["trueDamageTaken"]
            participants_row["true_dmg_dealt"] = row["trueDamageDealt"]
            participants_row["true_dmg_dealt_champ"] = row["trueDamageDealtToChampions"]
            
            participants_row["gold_earned"] = row["goldEarned"]
            participants_row["gold_spent"] = row["goldSpent"]
            
            
            participants.append(participants_row)
            
        games[j] = pd.DataFrame(participants)
        
        
            
        time.sleep(1)
        j += 1
        cont += 1
    except:
        cont += 1

In [30]:
games[0]

,champion,win,kills,deaths,assists,name,level,physical_dmg_taken,physical_dmg_dealt,physical_dmg_dealt_champ,magic_dmg_taken,magic_dmg_dealt,magic_dmg_dealt_champ,true_dmg_taken,true_dmg_dealt,true_dmg_dealt_champ,gold_earned,gold_spent
0,106,False,3,11,16,Aaronkorl,129,13047,11537,4146,12683,32447,8034,1910,959,842,9894,9850
1,39,False,5,15,16,Firsax,229,12520,34600,9921,19073,6987,3381,1900,0,0,10202,9650
2,164,False,7,13,18,yogurtsauce,511,12694,25263,13501,13304,711,711,4753,9019,6960,11899,11550
3,254,False,7,16,7,Asiangang69,75,10472,48138,16256,12872,0,0,969,838,713,11524,11350
4,105,False,13,12,12,vNMZOrDY,163,8045,5210,2482,13359,58626,21939,449,610,380,13728,13500
5,67,True,24,8,22,Herkalino,168,9761,63565,21152,10275,190,190,2771,7294,6434,15677,14340
6,8,True,11,6,42,IWantToNut,443,15022,2816,1091,9343,54833,30324,3206,1017,1017,12506,10385
7,236,True,15,10,23,Silweite,932,9809,62988,20901,6185,9013,5902,2481,1274,1274,13556,11625
8,61,True,13,7,41,That is Splendid,665,8839,2243,1115,4123,47966,26774,2268,833,833,12611,9900
9,267,True,4,4,50,Trap Star Life,230,6119,1871,1159,4139,13307,8100,168,184,184,11025,9235
